# ウェーブレット変換による音響解析

このノートブックでは、ウェーブレット変換を使用した音響信号の時間-周波数解析を学習します。
ウェーブレット変換は、FFTよりも時間分解能と周波数分解能のバランスが優れています。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pywt
from scipy import signal
from pydub import AudioSegment
from pydub.generators import Sine
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.family'] = 'DejaVu Sans'

## 1. ウェーブレット変換の基礎

In [ ]:
# チープ信号（周波数が時間とともに変化）の生成
fs = 1000  # サンプリング周波数
t = np.linspace(0, 2, fs * 2, False)

# 複数の周波数成分を持つ信号
f1 = 50  # Hz
f2 = 120  # Hz
f3 = 300  # Hz

# 異なる時間帯に異なる周波数
signal_data = np.zeros_like(t)
signal_data[0:fs//2] = np.sin(2 * np.pi * f1 * t[0:fs//2])  # 0-0.5秒: 50Hz
signal_data[fs//2:fs] = np.sin(2 * np.pi * f2 * t[fs//2:fs])  # 0.5-1秒: 120Hz
signal_data[fs:3*fs//2] = np.sin(2 * np.pi * f3 * t[fs:3*fs//2])  # 1-1.5秒: 300Hz
signal_data[3*fs//2:] = np.sin(2 * np.pi * f1 * t[3*fs//2:]) + np.sin(2 * np.pi * f2 * t[3*fs//2:])  # 1.5-2秒: 混合

# ノイズ追加
signal_data += 0.1 * np.random.randn(len(signal_data))

# 元信号の可視化
plt.figure(figsize=(12, 4))
plt.plot(t, signal_data)
plt.title('Original Signal - Frequency Changes Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

## 2. 連続ウェーブレット変換（CWT）

In [ ]:
# ウェーブレット変換の実行
wavelet = 'morl'  # Morletウェーブレット
scales = np.arange(1, 128)
coefficients, frequencies = pywt.cwt(signal_data, scales, wavelet, sampling_period=1/fs)

# スカログラムの表示
plt.figure(figsize=(15, 8))
plt.imshow(np.abs(coefficients), extent=[0, 2, frequencies[-1], frequencies[0]], 
           cmap='hot', aspect='auto', interpolation='bilinear')
plt.colorbar(label='Magnitude')
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (s)')
plt.title('Continuous Wavelet Transform (Scalogram)')
plt.ylim(0, 400)
plt.show()

## 3. 異なるウェーブレットの比較

In [ ]:
# 複数のウェーブレットでの解析
wavelets = ['morl', 'cgau1', 'mexh']
wavelet_names = ['Morlet', 'Complex Gaussian', 'Mexican Hat']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for i, (wavelet, name) in enumerate(zip(wavelets, wavelet_names)):
    # ウェーブレット関数の表示
    wavelet_func, _ = pywt.cwt(np.zeros(100), [10], wavelet)
    x = np.linspace(-2, 2, 100)
    
    if wavelet == 'morl':
        psi = np.exp(1j * 5 * x) * np.exp(-x**2 / 2)
        axes[0, i].plot(x, np.real(psi), label='Real')
        axes[0, i].plot(x, np.imag(psi), label='Imaginary')
        axes[0, i].legend()
    elif wavelet == 'mexh':
        psi = (2 / (np.sqrt(3) * np.pi**0.25)) * (1 - x**2) * np.exp(-x**2 / 2)
        axes[0, i].plot(x, psi)
    else:
        psi = np.exp(-x**2) * np.exp(1j * x)
        axes[0, i].plot(x, np.real(psi), label='Real')
        axes[0, i].plot(x, np.imag(psi), label='Imaginary')
        axes[0, i].legend()
    
    axes[0, i].set_title(f'{name} Wavelet')
    axes[0, i].grid(True)
    
    # ウェーブレット変換結果
    coeffs, freqs = pywt.cwt(signal_data, scales, wavelet, sampling_period=1/fs)
    
    im = axes[1, i].imshow(np.abs(coeffs), extent=[0, 2, freqs[-1], freqs[0]], 
                          cmap='hot', aspect='auto', interpolation='bilinear')
    axes[1, i].set_ylabel('Frequency (Hz)')
    axes[1, i].set_xlabel('Time (s)')
    axes[1, i].set_title(f'{name} CWT')
    axes[1, i].set_ylim(0, 400)
    
    plt.colorbar(im, ax=axes[1, i], label='Magnitude')

plt.tight_layout()
plt.show()

## 4. 音楽信号の解析

In [ ]:
# ピアノの和音をシミュレート
# C major chord (C4, E4, G4)
notes = [261.63, 329.63, 392.00]  # Hz
duration = 2000  # ms

# 各音符の生成
chord = AudioSegment.silent(duration=duration)
for freq in notes:
    note = Sine(freq).to_audio_segment(duration=duration)
    # エンベロープの適用
    note = note.fade_in(100).fade_out(500)
    chord = chord.overlay(note)

# 音楽的なフレーズの作成
sequence = AudioSegment.silent(duration=100)
melody_notes = [261.63, 293.66, 329.63, 349.23, 392.00, 440.00, 493.88, 523.25]  # C major scale

for i, freq in enumerate(melody_notes):
    note_duration = 300 + i * 50  # 音符の長さを変化
    note = Sine(freq).to_audio_segment(duration=note_duration)
    note = note.fade_in(50).fade_out(100)
    sequence += note + AudioSegment.silent(duration=50)

# 音楽信号をnumpy配列に変換
samples = np.array(sequence.get_array_of_samples())
if sequence.channels == 2:
    samples = samples.reshape((-1, 2))
    samples = samples.mean(axis=1)

samples = samples / np.max(np.abs(samples))  # 正規化
sample_rate = sequence.frame_rate
time_music = np.linspace(0, len(samples) / sample_rate, len(samples))

# 音楽信号の可視化
plt.figure(figsize=(15, 4))
plt.plot(time_music, samples)
plt.title('Musical Sequence Waveform')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)
plt.show()

In [ ]:
# 音楽信号のウェーブレット解析
# サンプル数を調整（計算時間短縮のため）
downsample_factor = 4
samples_down = samples[::downsample_factor]
fs_down = sample_rate // downsample_factor
time_down = time_music[::downsample_factor]

# ウェーブレット変換
scales_music = np.arange(1, 100)
coeffs_music, freqs_music = pywt.cwt(samples_down, scales_music, 'morl', 
                                    sampling_period=1/fs_down)

# 結果の可視化
plt.figure(figsize=(15, 10))

# スカログラム
plt.subplot(2, 1, 1)
plt.imshow(np.abs(coeffs_music), extent=[0, time_down[-1], freqs_music[-1], freqs_music[0]], 
           cmap='hot', aspect='auto', interpolation='bilinear')
plt.colorbar(label='Magnitude')
plt.ylabel('Frequency (Hz)')
plt.title('Musical Sequence - Wavelet Transform')
plt.ylim(0, 1000)

# 時間軸での平均パワー
plt.subplot(2, 1, 2)
power_over_time = np.mean(np.abs(coeffs_music)**2, axis=0)
plt.plot(time_down, power_over_time)
plt.title('Average Wavelet Power Over Time')
plt.xlabel('Time (s)')
plt.ylabel('Power')
plt.grid(True)

plt.tight_layout()
plt.show()

## 5. ウェーブレット・デノイジング

In [ ]:
# ノイズ除去の実演
# クリーンな信号の生成
t_clean = np.linspace(0, 1, 1000, False)
clean_signal = np.sin(2 * np.pi * 50 * t_clean) + 0.5 * np.sin(2 * np.pi * 120 * t_clean)

# ノイズを追加
noise = 0.5 * np.random.randn(len(clean_signal))
noisy_signal = clean_signal + noise

# 離散ウェーブレット変換によるデノイジング
# 多重解像度解析
wavelet_denoise = 'db4'  # Daubechies 4
coeffs = pywt.wavedec(noisy_signal, wavelet_denoise, level=6)

# 閾値処理
threshold = 0.1
coeffs_thresh = list(coeffs)
coeffs_thresh[1:] = [pywt.threshold(i, threshold, mode='soft') for i in coeffs_thresh[1:]]

# 信号の再構成
denoised_signal = pywt.waverec(coeffs_thresh, wavelet_denoise)

# 結果の比較
plt.figure(figsize=(15, 12))

plt.subplot(3, 1, 1)
plt.plot(t_clean, clean_signal)
plt.title('Original Clean Signal')
plt.ylabel('Amplitude')
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(t_clean, noisy_signal)
plt.title('Noisy Signal')
plt.ylabel('Amplitude')
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(t_clean, denoised_signal[:len(t_clean)])
plt.title('Denoised Signal (Wavelet Denoising)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.grid(True)

plt.tight_layout()
plt.show()

# ノイズ除去の性能評価
mse_noisy = np.mean((clean_signal - noisy_signal)**2)
mse_denoised = np.mean((clean_signal - denoised_signal[:len(clean_signal)])**2)

print(f"MSE (Noisy): {mse_noisy:.4f}")
print(f"MSE (Denoised): {mse_denoised:.4f}")
print(f"Improvement: {(mse_noisy - mse_denoised) / mse_noisy * 100:.2f}%")

## 6. ウェーブレット係数の詳細解析

In [ ]:
# 多重解像度解析の可視化
test_signal = noisy_signal
wavelet_analysis = 'db8'
levels = 5

# 分解
coeffs_detailed = pywt.wavedec(test_signal, wavelet_analysis, level=levels)

# 各レベルの再構成
fig, axes = plt.subplots(levels + 2, 1, figsize=(15, 3 * (levels + 2)))

# 元信号
axes[0].plot(test_signal)
axes[0].set_title('Original Signal')
axes[0].grid(True)

# 近似係数（最低周波数成分）
approx = pywt.upcoef('a', coeffs_detailed[0], wavelet_analysis, level=levels, 
                    take=len(test_signal))
axes[1].plot(approx)
axes[1].set_title(f'Approximation (Level {levels})')
axes[1].grid(True)

# 詳細係数（各周波数帯域）
for i in range(1, levels + 1):
    detail = pywt.upcoef('d', coeffs_detailed[i], wavelet_analysis, level=i, 
                        take=len(test_signal))
    axes[i + 1].plot(detail)
    axes[i + 1].set_title(f'Detail Level {i}')
    axes[i + 1].grid(True)

plt.tight_layout()
plt.show()

# 係数のエネルギー分布
energies = [np.sum(np.abs(c)**2) for c in coeffs_detailed]
labels = ['Approx'] + [f'Detail {i}' for i in range(1, levels + 1)]

plt.figure(figsize=(10, 6))
plt.bar(labels, energies)
plt.title('Energy Distribution Across Wavelet Coefficients')
plt.ylabel('Energy')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.show()